In [2]:
import pandas as pd
import numpy as np
import sklearn

from tardis.io.config_reader import Configuration

/Users/marcwilliamson/miniconda3/envs/numbatardis/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


In [118]:
def constructGrid(axesdict):
    axes = []
    dim = 1
    for key in axesdict:
        ax = axesdict[key]
        axes.append(ax)
        dim = dim * len(ax)
    axesmesh = np.meshgrid(*axes)
    tmp = np.dstack(axesmesh)
    gridpoints = tmp.reshape((dim, len(axes)), order='F')
    return gridpoints

class tardisGrid:
    
    def __init__(self, config, axesdict):
        try:
            tardis_config = Configuration.from_yaml(config)
        except TypeError:
            tardis_config = Configuration.from_config_dict(config)

        self.config = tardis_config
        self.axesdict = axesdict
        
        gridpoints = constructGrid(self.axesdict)
        self.grid = pd.DataFrame(data=gridpoints, columns=self.axesdict.keys())
        
        return
    
    def getconfigitem(self, key):
        keyitems = key.split('.')
        val = self.config[keyitems[0]]
        for k in keyitems[1:]:
            val = val[k]
        return val

In [119]:
axesdict = {'model.structure.velocity.start':np.arange(10000,15000,1000), 'model.abundances.He':np.arange(0,1,0.1),
           'model.abundances.H':np.arange(0,1,0.25)}

In [120]:
grid = tardisGrid('test.yml', axesdict)

In [121]:
grid.getconfigitem('model.structure.velocity.start')

<Quantity 10000. km / s>

In [122]:
grid.grid

,model.structure.velocity.start,model.abundances.He,model.abundances.H
0,10000.0,0.0,0.00
1,10000.0,0.1,0.00
2,10000.0,0.2,0.00
3,10000.0,0.3,0.00
4,10000.0,0.4,0.00
...,...,...,...
195,14000.0,0.5,0.75
196,14000.0,0.6,0.75
197,14000.0,0.7,0.75
198,14000.0,0.8,0.75
